1. Se importan todas las librerías necesarias para realizar nuestro ETL.

In [1]:
import pandas as pd
from pandas import json_normalize
import json as js
import ast as ast
import pyarrow as pa
import pyarrow.parquet as pq

2. Transformación de dataset "australian_users_reviews" a dataframe para manipulación con librería PANDAS.

In [2]:
list_rev = []
archivo1 = r'australian_user_reviews.json'
with open(archivo1, encoding='utf-8') as file:
    for line in file.readlines():
        list_rev.append(ast.literal_eval(line))

reviews = pd.DataFrame(list_rev)

3. Identificamos que la columna 'reviews' contiene una lista de elementos para cada registro, con el método explode, vamos a expandir esta información.

In [3]:
reviews = reviews.explode('reviews',ignore_index=True)

4. Se procede a aplicar el método de normalización de datos, que termina por darle un formato tabular aplanado a esta columna del dataframe principal.

In [4]:
reviews_expanded = pd.json_normalize(reviews['reviews']) #Creamos un dataframe que recibe los datos de la columna review normalizada
reviews_expanded = reviews_expanded.replace('',None)  #Reemplazamos los espacios vacios por None


5. Se realiza un join entre los dataframe reviews y reviews_expanded y se elimina la columna items que contenía lo que ahora contiene la columna "reviews".

In [5]:
reviews = reviews.join(reviews_expanded)
reviews.drop(columns=['reviews'],inplace=True) #Se elimina a través del método drop la columna reviews, ya que con el join pasado, se sumaron los datos de la columna pero de manera aplanada.

6. Se da un vistazo a cómo queda el dataframe aplanado.

In [6]:
reviews

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,None,"Posted November 5, 2011.",None,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,None,"Posted July 15, 2011.",None,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,None,"Posted April 21, 2011.",None,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,None,"Posted June 24, 2014.",None,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,None,"Posted September 8, 2013.",None,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...,...
59328,76561198312638244,http://steamcommunity.com/profiles/76561198312...,None,Posted July 10.,None,70,No ratings yet,True,a must have classic from steam definitely wort...
59329,76561198312638244,http://steamcommunity.com/profiles/76561198312...,None,Posted July 8.,None,362890,No ratings yet,True,this game is a perfect remake of the original ...
59330,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,1 person found this review funny,Posted July 3.,None,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,None,Posted July 20.,None,730,No ratings yet,True,:D


7. Se revisan datos básicos como la cantidad de filas, los nombres de las columnas y los tipos de valores dentro de cada columna.

In [7]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59333 non-null  object
 1   user_url     59333 non-null  object
 2   funny        8151 non-null   object
 3   posted       59305 non-null  object
 4   last_edited  6140 non-null   object
 5   item_id      59305 non-null  object
 6   helpful      59305 non-null  object
 7   recommend    59305 non-null  object
 8   review       59275 non-null  object
dtypes: object(9)
memory usage: 4.1+ MB


8. Se dispondrá a eliminar columnas del dataframe  que no sean necesarias para el análisis como por ejemplo, las columnas "user_url, ya que toda la información útil ya esta scrapeada en la tabla, "Funny" ya que es solo una reacción que se genera al reaccionar a través de un botón en la página de steam, "Posted" tampoco va a ser requerido para las funciones propuestas, al igual que "last_edited" y "helpful".

In [8]:
reviews.drop(columns=['user_url','funny','posted','last_edited','helpful'],inplace=True)

#Se eliminan las columnas mencionadas en el argumento y se reemplaza el dataframe con estos cambios.

In [9]:
reviews

,user_id,item_id,recommend,review
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,22200,True,It's unique and worth a playthrough.
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,251610,True,I know what you think when you see this title ...
4,js41637,227300,True,For a simple (it's actually not all that simpl...
...,...,...,...,...
59328,76561198312638244,70,True,a must have classic from steam definitely wort...
59329,76561198312638244,362890,True,this game is a perfect remake of the original ...
59330,LydiaMorley,273110,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,730,True,:D


9. Se realiza cambio de tipo de dato de la columna 'item_id' y 'recommend' a flotante

In [10]:
reviews['item_id']=reviews['item_id'].astype(float)

In [11]:
reviews['recommend']=reviews['recommend'].astype(bool)

10. Importamos los dataframes contenidos en formato .parquet para realizar la adición requerida.

In [12]:
games_def = pd.read_parquet('ETL.GAMES.parquet')

In [13]:
items = pd.read_parquet('ETL.ITEMS.parquet')

11. Comenzamos a realizar el merge, el cual me permitirá complementar los datasets que tengo con información que tengan las otras tablas por medio  de la llave primaria. Para ello, utilizamos el siguiente comando:

In [14]:
reviews = reviews.merge(games_def[['item_name','item_id']].notnull(), on='item_id', how='left')

In [15]:
reviews

,user_id,item_id,recommend,review,item_name
0,76561197970982479,1250.0,True,Simple yet with great replayability. In my opi...,NaN
1,76561197970982479,22200.0,True,It's unique and worth a playthrough.,NaN
2,76561197970982479,43110.0,True,Great atmosphere. The gunplay can be a bit chu...,NaN
3,js41637,251610.0,True,I know what you think when you see this title ...,NaN
4,js41637,227300.0,True,For a simple (it's actually not all that simpl...,NaN
...,...,...,...,...,...
59328,76561198312638244,70.0,True,a must have classic from steam definitely wort...,NaN
59329,76561198312638244,362890.0,True,this game is a perfect remake of the original ...,NaN
59330,LydiaMorley,273110.0,True,had so much fun plaing this and collecting res...,NaN
59331,LydiaMorley,730.0,True,:D,NaN


In [16]:
items_unicos = items.drop(items[items.duplicated(subset='item_id')].index)

In [17]:
items_unicos

,user_id,items_count,item_id,item_name,playtime_forever
0,76561197970982479,277,10.0,Counter-Strike,0.100000
1,76561197970982479,277,20.0,Team Fortress Classic,0.000000
2,76561197970982479,277,30.0,Day of Defeat,0.116667
3,76561197970982479,277,40.0,Deathmatch Classic,0.000000
4,76561197970982479,277,50.0,Half-Life: Opposing Force,0.000000
...,...,...,...,...,...
4913278,76561198081168593,84,375450.0,NOBUNAGA'S AMBITION: Souzou SengokuRisshiden,0.850000
5092969,76561198101994484,139,353390.0,Alienware Steam Machine,0.000000
5105048,76561198107283457,33,354280.0,ChaosTower,2.733333
5119454,inven,3603,433920.0,Aveyond 4: Shadow Of The Mist,0.000000


In [18]:
reviews = reviews.merge(items_unicos[['item_id','item_name']], on='item_id', how='left')

In [19]:
reviews

,user_id,item_id,recommend,review,item_name_x,item_name_y
0,76561197970982479,1250.0,True,Simple yet with great replayability. In my opi...,NaN,Killing Floor
1,76561197970982479,22200.0,True,It's unique and worth a playthrough.,NaN,Zeno Clash
2,76561197970982479,43110.0,True,Great atmosphere. The gunplay can be a bit chu...,NaN,Metro 2033
3,js41637,251610.0,True,I know what you think when you see this title ...,NaN,Barbie™ Dreamhouse Party™
4,js41637,227300.0,True,For a simple (it's actually not all that simpl...,NaN,Euro Truck Simulator 2
...,...,...,...,...,...,...
59328,76561198312638244,70.0,True,a must have classic from steam definitely wort...,NaN,Half-Life
59329,76561198312638244,362890.0,True,this game is a perfect remake of the original ...,NaN,Black Mesa
59330,LydiaMorley,273110.0,True,had so much fun plaing this and collecting res...,NaN,Counter-Strike Nexon: Zombies
59331,LydiaMorley,730.0,True,:D,NaN,Counter-Strike: Global Offensive


In [20]:
reviews['item_name'] = reviews['item_name_x'].combine_first(reviews['item_name_y'])

In [21]:
reviews.drop(columns=['item_name_x','item_name_y'],inplace=True)

In [22]:
reviews

,user_id,item_id,recommend,review,item_name
0,76561197970982479,1250.0,True,Simple yet with great replayability. In my opi...,Killing Floor
1,76561197970982479,22200.0,True,It's unique and worth a playthrough.,Zeno Clash
2,76561197970982479,43110.0,True,Great atmosphere. The gunplay can be a bit chu...,Metro 2033
3,js41637,251610.0,True,I know what you think when you see this title ...,Barbie™ Dreamhouse Party™
4,js41637,227300.0,True,For a simple (it's actually not all that simpl...,Euro Truck Simulator 2
...,...,...,...,...,...
59328,76561198312638244,70.0,True,a must have classic from steam definitely wort...,Half-Life
59329,76561198312638244,362890.0,True,this game is a perfect remake of the original ...,Black Mesa
59330,LydiaMorley,273110.0,True,had so much fun plaing this and collecting res...,Counter-Strike Nexon: Zombies
59331,LydiaMorley,730.0,True,:D,Counter-Strike: Global Offensive


In [23]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    59333 non-null  object
 1   item_id    59305 non-null  object
 2   recommend  59333 non-null  bool  
 3   review     59275 non-null  object
 4   item_name  52781 non-null  object
dtypes: bool(1), object(4)
memory usage: 1.9+ MB


12. Eliminamos las filas que contengan valores  nulos

In [24]:
reviews = reviews.dropna().reset_index(drop=True)
reviews

,user_id,item_id,recommend,review,item_name
0,76561197970982479,1250.0,True,Simple yet with great replayability. In my opi...,Killing Floor
1,76561197970982479,22200.0,True,It's unique and worth a playthrough.,Zeno Clash
2,76561197970982479,43110.0,True,Great atmosphere. The gunplay can be a bit chu...,Metro 2033
3,js41637,251610.0,True,I know what you think when you see this title ...,Barbie™ Dreamhouse Party™
4,js41637,227300.0,True,For a simple (it's actually not all that simpl...,Euro Truck Simulator 2
...,...,...,...,...,...
52753,76561198312638244,130.0,True,if you liked Half life i would really recommen...,Half-Life: Blue Shift
52754,76561198312638244,70.0,True,a must have classic from steam definitely wort...,Half-Life
52755,76561198312638244,362890.0,True,this game is a perfect remake of the original ...,Black Mesa
52756,LydiaMorley,273110.0,True,had so much fun plaing this and collecting res...,Counter-Strike Nexon: Zombies


In [25]:
reviews.to_parquet('ETL.REVIEWS.parquet', index=False)

# dentro del argumento el index = False,  
# se indica que no se debe incluir la columna de índice en el archivo parquet.